In [1]:
import pandas as pd
import json

from datetime import datetime

## Legislator Data

https://github.com/unitedstates/congress-legislators?tab=readme-ov-file

In [2]:
drop = [ 'url', 'address', 'phone', 'contact_form', 'rss_url', 'twitter', 'twitter_id', 'facebook', 'youtube', 'youtube_id',
        'thomas_id', 'opensecrets_id', 'lis_id', 'fec_ids', 'cspan_id', 'mastodon','votesmart_id', 'ballotpedia_id','washington_post_id',]

In [3]:
legislators_current = pd.read_csv('/content/drive/MyDrive/UMBC Fall 2023/DATA606 - Capstone/senator data/legislators-current.csv', parse_dates=['birthday'])
legislators_current = legislators_current.drop(columns=drop)
legislators_historical = pd.read_csv('/content/drive/MyDrive/UMBC Fall 2023/DATA606 - Capstone/senator data/legislators-historical.csv', parse_dates=['birthday'])
legislators_historical = legislators_historical.drop(columns=drop)

In [4]:
legislators = pd.concat([legislators_current, legislators_historical], ignore_index=True, axis=0)

In [5]:
legislators['type'].unique()

array(['sen', 'rep'], dtype=object)

In [6]:
legislators.columns

Index(['last_name', 'first_name', 'middle_name', 'suffix', 'nickname',
       'full_name', 'birthday', 'gender', 'type', 'state', 'district',
       'senate_class', 'party', 'bioguide_id', 'govtrack_id', 'icpsr_id',
       'wikipedia_id'],
      dtype='object')

In [44]:
senators = legislators[legislators['type']=='sen']

In [46]:
senators['name'] = senators['first_name'].str.casefold().str.replace('[^a-zA-Z]', '') + senators['last_name'].str.casefold().str.replace('[^a-zA-Z]', '')

<ipython-input-46-fb85ee062e41>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  senators['name'] = senators['first_name'].str.casefold().str.replace('[^a-zA-Z]', '') + senators['last_name'].str.casefold().str.replace('[^a-zA-Z]', '')
<ipython-input-46-fb85ee062e41>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senators['name'] = senators['first_name'].str.casefold().str.replace('[^a-zA-Z]', '') + senators['last_name'].str.casefold().str.replace('[^a-zA-Z]', '')


In [47]:
len(senators['name'].unique())

1895

In [48]:
senators = senators.drop_duplicates('name')

## Job Approval Ratings

https://github.com/fivethirtyeight/data/tree/master/polls

2016-11-17 - 2024-03-15



In [50]:
senate_polls = pd.read_csv('/content/drive/MyDrive/UMBC Fall 2023/DATA606 - Capstone/senator data/senate_polls.csv', parse_dates=['start_date','end_date','election_date'])
senate_polls = senate_polls.drop(columns=['endorsed_candidate_id','endorsed_candidate_name','endorsed_candidate_party'])
senate_polls_historical = pd.read_csv('/content/drive/MyDrive/UMBC Fall 2023/DATA606 - Capstone/senator data/senate_polls_historical.csv', parse_dates=['start_date','end_date','election_date'])

In [51]:
polls = pd.concat([senate_polls_historical, senate_polls], ignore_index=True, axis=0)
polls = polls.drop(columns=['subpopulation'])

In [52]:
polls.columns

Index(['poll_id', 'pollster_id', 'pollster', 'sponsor_ids', 'sponsors',
       'display_name', 'pollster_rating_id', 'pollster_rating_name',
       'fte_grade', 'methodology', 'transparency_score', 'state', 'start_date',
       'end_date', 'sponsor_candidate_id', 'sponsor_candidate',
       'sponsor_candidate_party', 'question_id', 'sample_size', 'population',
       'population_full', 'tracking', 'created_at', 'notes', 'url', 'source',
       'internal', 'partisan', 'race_id', 'cycle', 'office_type',
       'seat_number', 'seat_name', 'election_date', 'stage',
       'nationwide_batch', 'ranked_choice_reallocated', 'ranked_choice_round',
       'party', 'answer', 'candidate_id', 'candidate_name', 'pct'],
      dtype='object')

In [54]:
honorificPrefix=r'(\,|\.|Ms|Mr)'
honorificSuffix=r'(\,|\.|Jr.|III|II|IV)'

In [55]:
polls['candidate_name'] = polls['candidate_name'].replace(honorificPrefix,'',regex=True)
polls['candidate_name'] = polls['candidate_name'].replace(honorificSuffix,'',regex=True)

In [56]:
polls['first_name'] = polls['candidate_name'].str.split().str[0]
polls['last_name'] = polls['candidate_name'].str.split().str[-1]

In [57]:
polls['name'] = polls['first_name'].str.casefold().str.replace('[^a-zA-Z]','') + polls['last_name'].str.casefold().str.replace('[^a-zA-Z]', '')

<ipython-input-57-8aaba56868fa>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  polls['name'] = polls['first_name'].str.casefold().str.replace('[^a-zA-Z]','') + polls['last_name'].str.casefold().str.replace('[^a-zA-Z]', '')


In [58]:
len(polls['name'].unique())

485

## Merge Senator and Job Approval Data
Challenges:
- Non-normalized names (aka variations in spelling, structure, and nicknames used)


In [143]:
df = pd.merge(polls, senators,  how='inner', left_on=['name'], right_on = ['name'])

In [60]:
df.columns

Index(['poll_id', 'pollster_id', 'pollster', 'sponsor_ids', 'sponsors',
       'display_name', 'pollster_rating_id', 'pollster_rating_name',
       'fte_grade', 'methodology', 'transparency_score', 'state_x',
       'start_date', 'end_date', 'sponsor_candidate_id', 'sponsor_candidate',
       'sponsor_candidate_party', 'question_id', 'sample_size', 'population',
       'population_full', 'tracking', 'created_at', 'notes', 'url', 'source',
       'internal', 'partisan', 'race_id', 'cycle', 'office_type',
       'seat_number', 'seat_name', 'election_date', 'stage',
       'nationwide_batch', 'ranked_choice_reallocated', 'ranked_choice_round',
       'party_x', 'answer', 'candidate_id', 'candidate_name', 'pct',
       'first_name_x', 'last_name_x', 'name', 'last_name_y', 'first_name_y',
       'middle_name', 'suffix', 'nickname', 'full_name', 'birthday', 'gender',
       'type', 'state_y', 'district', 'senate_class', 'party_y', 'bioguide_id',
       'govtrack_id', 'icpsr_id', 'wikipedia_i

In [64]:
len(df['full_name'].unique())

96

In [65]:
print(list(df['full_name'].unique()))

['Raphael G. Warnock', 'Mark Kelly', 'John Fetterman', 'Ron Johnson', 'Todd Young', 'Eric Schmitt', 'Michael F. Bennet', 'James Lankford', 'Markwayne Mullin', 'Ted Budd', 'Marco Rubio', 'Tammy Duckworth', 'Charles E. Schumer', 'Ron Wyden', 'Patty Murray', 'John Boozman', 'Mike Lee', 'Katie Boyd Britt', 'Jerry Moran', 'Richard Blumenthal', 'Peter Welch', 'Lisa Murkowski', 'John Thune', 'John Kennedy', 'Tim Scott', 'Rand Paul', 'Kelly Ayotte', 'Kelly Loeffler', 'John McCain', 'Roy Blunt', 'Jon Ossoff', 'David Perdue', 'Joni Ernst', 'Lindsey Graham', 'Steve Daines', 'Susan M. Collins', 'Martha McSally', 'John Cornyn', 'Richard J. Durbin', 'Gary C. Peters', 'Tina Smith', 'Cory A. Booker', 'John W. Hickenlooper', 'Cory Gardner', 'Mark R. Warner', 'Tommy Tuberville', 'Roger Marshall', 'Cindy Hyde-Smith', 'Mitch McConnell', 'Bill Hagerty', 'Edward J. Markey', 'Ben Ray Luján', 'Dan Sullivan', 'Jeanne Shaheen', 'Cynthia M. Lummis', 'Tom Cotton', 'James M. Inhofe', 'Shelley Moore Capito', 'Jeff 

In [67]:
df['end_date'].min()

Timestamp('2016-11-17 00:00:00')

### Determine columns to keep for initial modeling
Potentially interesting
- senate_class vs. seat_name
- poll_id vs. pollster_id
- stage

In [144]:
df = df.dropna(axis='columns', how='any')

In [145]:
drop = ['bioguide_id', 'party_x', 'state_y','type', 'name','candidate_name','poll_id', 'pollster_id', 'pollster', 'display_name',
        'pollster_rating_id', 'pollster_rating_name','question_id','seat_number','created_at','race_id',
        'office_type','nationwide_batch','candidate_id', 'answer', 'seat_name',
       'govtrack_id', 'wikipedia_id', 'first_name_x','first_name_y','last_name_x','last_name_y']

In [154]:
df = df.rename(columns={'state_x':'state','party_y':'party'})

In [164]:
df = df[['full_name', 'pct', 'birthday', 'gender','party','senate_class','stage','cycle', 'start_date', 'end_date', 'ranked_choice_reallocated']]

In [168]:
df

,full_name,pct,birthday,gender,party,senate_class,stage,cycle,start_date,end_date,ranked_choice_reallocated
0,Raphael G. Warnock,51.1,1969-07-23,M,Democrat,3.0,runoff,2022,2022-12-03,2022-12-05,False
1,Raphael G. Warnock,51.0,1969-07-23,M,Democrat,3.0,runoff,2022,2022-12-01,2022-12-05,False
2,Raphael G. Warnock,50.5,1969-07-23,M,Democrat,3.0,runoff,2022,2022-12-04,2022-12-04,False
3,Raphael G. Warnock,50.0,1969-07-23,M,Democrat,3.0,runoff,2022,2022-12-04,2022-12-04,False
4,Raphael G. Warnock,52.2,1969-07-23,M,Democrat,3.0,runoff,2022,2022-12-04,2022-12-04,False
...,...,...,...,...,...,...,...,...,...,...,...
3097,Benjamin L. Cardin,49.4,1943-10-05,M,Democrat,1.0,general,2018,2018-10-01,2018-10-06,False
3098,Benjamin L. Cardin,56.0,1943-10-05,M,Democrat,1.0,general,2018,2018-09-11,2018-09-16,False
3099,Thomas R. Carper,61.0,1947-01-23,M,Democrat,1.0,general,2018,2018-09-11,2018-09-17,False
3100,Thomas R. Carper,60.0,1947-01-23,M,Democrat,1.0,general,2018,2018-09-11,2018-09-17,False


In [169]:
df.to_csv('/content/drive/MyDrive/UMBC Fall 2023/DATA606 - Capstone/senator data/cleaned_data.csv')

In [80]:
#.groupby('name').count()

## Pulling Senator data from the 114th (2015) to the 118th (2025) Congress.
- Could use this to get which congress number each senator belongs to?
- Is there any way to get the biographies programmatically?

https://bioguide.congress.gov/

In [26]:
senators2 = pd.read_csv("/content/drive/MyDrive/UMBC Fall 2023/DATA606 - Capstone/senator data/senators_114_118_congress.csv", parse_dates=['birthYear','deathYear'])
senators2 = senators2.drop(columns=['unaccentedMiddleName','honorificTitle'])

In [27]:
senators2.columns

Index(['id', 'givenName', 'familyName', 'middleName', 'unaccentedGivenName',
       'unaccentedFamilyName', 'nickName', 'honorificPrefix',
       'honorificSuffix', 'birthYear', 'deathYear', 'congresses'],
      dtype='object')

In [28]:
senators2.iloc[0]['congresses']

'[{"position":"Senator","congressNumber":108,"stateName":"TN","parties":["Republican"]},{"position":"Senator","congressNumber":109,"stateName":"TN","parties":["Republican"]},{"position":"Senator","congressNumber":110,"stateName":"TN","parties":["Republican"]},{"position":"Senator","congressNumber":111,"stateName":"TN","parties":["Republican"]},{"position":"Senator","congressNumber":112,"stateName":"TN","parties":["Republican"]},{"position":"Senator","congressNumber":113,"stateName":"TN","parties":["Republican"]},{"position":"Senator","congressNumber":114,"stateName":"TN","parties":["Republican"]},{"position":"Senator","congressNumber":115,"stateName":"TN","parties":["Republican"]},{"position":"Senator","congressNumber":116,"stateName":"TN","parties":["Republican"]}]'

In [29]:
# TODO: come up with clever way to iterate through congresses each has been in
#senators.join([senators['congresses'].apply(json.loads).apply(pd.Series))

In [30]:
senators2['name'] = senators2['unaccentedGivenName'].str.casefold().str.replace('[^a-zA-Z]', ' ') + ' ' + senators2['unaccentedFamilyName'].str.casefold().str.replace('[^a-zA-Z]', ' ')

<ipython-input-30-65f94c304f36>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  senators2['name'] = senators2['unaccentedGivenName'].str.casefold().str.replace('[^a-zA-Z]', ' ') + ' ' + senators2['unaccentedFamilyName'].str.casefold().str.replace('[^a-zA-Z]', ' ')


In [31]:
df2 = pd.merge(senators2, senators,  how='inner', left_on=['name'], right_on = ['name'])

In [32]:
df2['name']

Series([], Name: name, dtype: object)

In [33]:
len(senators2['name'].unique())

198

## Old (1950's - 2009) Job Approval Ratings Data

### JARS for U.S. Senators 1978-2009

https://jmj313.web.lehigh.edu/node/6

In [ ]:
jars = pd.read_excel("/content/drive/MyDrive/UMBC Fall 2023/DATA606 - Capstone/senator data/JARs_sen--032910.xls")

In [ ]:
jars = jars.drop(columns=['OFFICE','SOURCED','Unnamed: 24', 'DAYIN', 'MONTHIN', 'YEARIN', 'DAYOUT', 'MONTHOUT', 'YEAROUT'])

In [ ]:
jars

In [ ]:
# Drop all that are not "1 - Standard Job Performance Question"
jars['QUESTION'].value_counts()

In [ ]:
jars = jars[jars['QUESTION'] == 1.0] # 5819 rows
jars = jars.drop(columns=['QUESTION'])

In [ ]:
jars

In [ ]:
len(jars['NAME'].unique())

TODO:
* Clean up Senators dataframe
* Clean up JARS dataframe
* Merge Senators and JARS dataframe